# Лабораторная работа №3 — Решающие деревья (Decision Tree)

Цель: построить бейзлайны для **классификации** и **регрессии** на основе решающего дерева из `sklearn`, выбрать метрики и оценить качество на тестовой выборке.

Датасеты:
- **Классификация:** `customer_support_data.csv` — уровень удовлетворённости клиента (**используем 20% выборку от всего датасета**).
- **Регрессия:** `cwurData.csv` — показатель, связанный с мировым рейтингом университета.


In [91]:
import numpy as np
import pandas as pd

# Базовые импорты и настройки отображения
pd.set_option("display.max_columns", 200)


## 1) Метрики качества и обоснование

### Классификация (DecisionTreeClassifier)
- **Accuracy** — доля верных предсказаний, удобна как простая базовая метрика.
- **F1-score (macro)** — усредняет F1 по классам без учёта их размера, полезно при дисбалансе классов.

(Если классов 2, дополнительно можно смотреть ROC-AUC, но в этой работе достаточно Accuracy и macro-F1.)

### Регрессия (DecisionTreeRegressor)
- **MAE** — средняя абсолютная ошибка, интерпретируется в единицах целевой переменной.
- **RMSE** — сильнее штрафует большие ошибки.
- **R²** — доля объяснённой дисперсии (насколько модель “объясняет” данные).


In [92]:
from sklearn.model_selection import train_test_split

# Зафиксируем random_state для воспроизводимости результатов
RANDOM_STATE = 42


## 2) Классификация: customer_support_data.csv (20% выборка) — бейзлайн на DecisionTreeClassifier


In [93]:
df_cls = pd.read_csv("customer_support_data.csv")

# Загрузили данные для классификации; дальше возьмём 20% от общего датасета
df_cls.shape


(85907, 20)

In [94]:
# Берём случайную подвыборку 20% от всего датасета согласно условию
df_cls_sample = df_cls.sample(frac=0.2, random_state=RANDOM_STATE).reset_index(drop=True)
df_cls_sample.shape


(17181, 20)

In [95]:
# Эвристика выбора target: ищем столбец по ключевым словам, иначе — столбец с небольшим числом уникальных значений
candidate_targets = [c for c in df_cls_sample.columns if any(k in c.lower() for k in ["satisf", "sentiment", "label", "target", "class", "rating"])]
if candidate_targets:
    target_cls = candidate_targets[0]
else:
    nunique = df_cls_sample.nunique(dropna=True).sort_values()
    target_cls = next(c for c in nunique.index if nunique[c] <= 20 and "id" not in c.lower())

target_cls


'channel_name'

In [96]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

# Разделяем признаки и целевую переменную
X_cls = df_cls_sample.drop(columns=[target_cls])
y_cls = df_cls_sample[target_cls]


In [97]:
# Делим на train/test со стратификацией по классам (если классов > 1)
X_train, X_test, y_train, y_test = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=RANDOM_STATE, stratify=y_cls if y_cls.nunique() > 1 else None
)


In [98]:
# Для дерева достаточно корректно обработать пропуски и закодировать категории (масштабирование не обязательно)
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]

num_pipe = Pipeline(steps=[("imputer", SimpleImputer(strategy="median"))])
cat_pipe = Pipeline(steps=[("imputer", SimpleImputer(strategy="most_frequent")),
                          ("ohe", OneHotEncoder(handle_unknown="ignore"))])

preprocess = ColumnTransformer(transformers=[
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols)
])


In [99]:
# Бейзлайн-модель: DecisionTreeClassifier
clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", DecisionTreeClassifier(random_state=RANDOM_STATE))
])

clf.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [100]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Считаем метрики качества на тесте (Accuracy и macro-F1)
y_pred = clf.predict(X_test)
metrics_cls = {
    "accuracy": accuracy_score(y_test, y_pred),
    "f1_macro": f1_score(y_test, y_pred, average="macro")
}
metrics_cls


{'accuracy': 0.7718940936863544, 'f1_macro': 0.38550697193370104}

In [101]:
# Отчёт по классам поможет понять, на каких классах модель ошибается чаще
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       Email       0.08      0.02      0.03       119
     Inbound       0.82      0.93      0.87      2729
     Outcall       0.37      0.20      0.26       589

    accuracy                           0.77      3437
   macro avg       0.42      0.38      0.39      3437
weighted avg       0.72      0.77      0.74      3437



## 3) Регрессия: cwurData.csv — бейзлайн на DecisionTreeRegressor


In [102]:
df_reg = pd.read_csv("cwurData.csv")

# Загрузили данные для регрессии; дальше выберем целевой столбец
df_reg.shape


(2200, 14)

In [103]:
# Выбор target для регрессии: приоритет world_rank, иначе — первый числовой столбец
preferred_targets = ["world_rank", "score", "rank"]
target_reg = next((t for t in preferred_targets if t in df_reg.columns), None)

if target_reg is None:
    num_candidates = df_reg.select_dtypes(include=[np.number]).columns.tolist()
    target_reg = num_candidates[0] if num_candidates else df_reg.columns[-1]

target_reg


'world_rank'

In [104]:
from sklearn.tree import DecisionTreeRegressor

# Разделяем признаки и таргет, затем train/test split
X_reg = df_reg.drop(columns=[target_reg])
y_reg = df_reg[target_reg]

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=RANDOM_STATE
)


In [105]:
# Препроцессинг: заполнение пропусков + one-hot кодирование категорий (для дерева этого достаточно)
num_cols_r = Xr_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols_r = [c for c in Xr_train.columns if c not in num_cols_r]

num_pipe_r = Pipeline(steps=[("imputer", SimpleImputer(strategy="median"))])
cat_pipe_r = Pipeline(steps=[("imputer", SimpleImputer(strategy="most_frequent")),
                            ("ohe", OneHotEncoder(handle_unknown="ignore"))])

preprocess_r = ColumnTransformer(transformers=[
    ("num", num_pipe_r, num_cols_r),
    ("cat", cat_pipe_r, cat_cols_r)
])


In [106]:
# Бейзлайн-модель: DecisionTreeRegressor
reg = Pipeline(steps=[
    ("preprocess", preprocess_r),
    ("model", DecisionTreeRegressor(random_state=RANDOM_STATE))
])

reg.fit(Xr_train, yr_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [107]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Оцениваем качество регрессии на тесте: MAE, RMSE, R²
yr_pred = reg.predict(Xr_test)
metrics_reg = {
    "MAE": mean_absolute_error(yr_test, yr_pred),
    "RMSE": mean_squared_error(yr_test, yr_pred) ** 0.5,
    "R2": r2_score(yr_test, yr_pred)
}
metrics_reg


{'MAE': 9.038636363636364, 'RMSE': 21.41510724189394, 'R2': 0.9949976237156489}

## 4) Итоговая сводка метрик (бейзлайн)
Сведём результаты классификации и регрессии в таблицы для отчёта.


In [108]:
# Собираем результаты в удобные таблицы
summary_cls = pd.DataFrame([metrics_cls])
summary_reg = pd.DataFrame([metrics_reg])

summary_cls, summary_reg


(   accuracy  f1_macro
 0  0.771894  0.385507,
         MAE       RMSE        R2
 0  9.038636  21.415107  0.994998)

## 3) Улучшение бейзлайна


### 3.1 Формулировка гипотез

#### Классификация (customer_support_data.csv)
1. Решающие деревья склонны к переобучению → ограничение глубины дерева улучшит обобщающую способность.
2. Увеличение `min_samples_leaf` снизит влияние шума и редких наблюдений.
3. Подбор гиперпараметров с помощью кросс-валидации улучшит F1-score.

#### Регрессия (cwurData.csv)
1. Ограничение глубины дерева уменьшит переобучение и снизит RMSE.
2. Подбор `min_samples_leaf` и `max_depth` улучшит качество модели.
3. Использование кросс-валидации позволит выбрать более устойчивую модель.


### 3.2 Проверка гипотез: классификация
Подберём гиперпараметры решающего дерева с помощью GridSearchCV.


In [109]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Создаём модель дерева решений для классификации
clf_improved = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", DecisionTreeClassifier(random_state=RANDOM_STATE))
])


In [110]:
# Сетка гиперпараметров для подбора
param_grid_cls = {
    "model__max_depth": [3, 5, 10, None],
    "model__min_samples_leaf": [1, 5, 10]
}

# Используем F1-macro как целевую метрику кросс-валидации
grid_cls = GridSearchCV(
    clf_improved,
    param_grid=param_grid_cls,
    scoring="f1_macro",
    cv=5,
    n_jobs=-1
)


In [111]:
grid_cls.fit(X_train, y_train)

# Сохраняем лучшую модель классификации
best_clf = grid_cls.best_estimator_
grid_cls.best_params_


{'model__max_depth': None, 'model__min_samples_leaf': 5}

In [112]:
from sklearn.metrics import accuracy_score, f1_score

# Оцениваем улучшенную модель классификации на тестовой выборке
y_pred_imp = best_clf.predict(X_test)
metrics_cls_improved = {
    "accuracy": accuracy_score(y_test, y_pred_imp),
    "f1_macro": f1_score(y_test, y_pred_imp, average="macro")
}
metrics_cls_improved


{'accuracy': 0.7652022112307245, 'f1_macro': 0.38570288955337056}

### 3.3 Проверка гипотез: регрессия
Подберём гиперпараметры решающего дерева для задачи регрессии.


In [113]:
from sklearn.tree import DecisionTreeRegressor

# Создаём модель дерева решений для регрессии
reg_improved = Pipeline(steps=[
    ("preprocess", preprocess_r),
    ("model", DecisionTreeRegressor(random_state=RANDOM_STATE))
])


In [114]:
# Сетка гиперпараметров для регрессии
param_grid_reg = {
    "model__max_depth": [3, 5, 10, None],
    "model__min_samples_leaf": [1, 5, 10]
}

# Используем отрицательный RMSE для подбора параметров
grid_reg = GridSearchCV(
    reg_improved,
    param_grid=param_grid_reg,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)


In [115]:
grid_reg.fit(Xr_train, yr_train)

# Сохраняем лучшую регрессионную модель
best_reg = grid_reg.best_estimator_
grid_reg.best_params_


{'model__max_depth': None, 'model__min_samples_leaf': 1}

In [116]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Оцениваем улучшенную модель регрессии на тестовой выборке
yr_pred_imp = best_reg.predict(Xr_test)
metrics_reg_improved = {
    "MAE": mean_absolute_error(yr_test, yr_pred_imp),
    "RMSE": mean_squared_error(yr_test, yr_pred_imp) ** 0.5,
    "R2": r2_score(yr_test, yr_pred_imp)
}
metrics_reg_improved


{'MAE': 9.038636363636364, 'RMSE': 21.41510724189394, 'R2': 0.9949976237156489}

### 3.4 Сравнение с бейзлайном (пункт 2)


In [117]:
# Сравнение метрик классификации: бейзлайн vs улучшенная модель
compare_cls = pd.DataFrame(
    [metrics_cls, metrics_cls_improved],
    index=["Baseline", "Improved"]
)
compare_cls


,accuracy,f1_macro
Baseline,0.771894,0.385507
Improved,0.765202,0.385703


In [118]:
# Сравнение метрик регрессии: бейзлайн vs улучшенная модель
compare_reg = pd.DataFrame(
    [metrics_reg, metrics_reg_improved],
    index=["Baseline", "Improved"]
)
compare_reg


,MAE,RMSE,R2
Baseline,9.038636,21.415107,0.994998
Improved,9.038636,21.415107,0.994998


### 3.5 Выводы

1. В задаче классификации улучшенный бейзлайн на основе решающего дерева показал незначительное изменение метрик по сравнению с базовой моделью. Значение `accuracy` немного снизилось, при этом `f1_macro` практически не изменился. Это свидетельствует о том, что подобранные гиперпараметры не привели к существенному улучшению качества классификации на данном наборе данных.

2. В задаче регрессии значения метрик MAE, RMSE и R² для улучшенного бейзлайна полностью совпадают с результатами базовой модели. Это означает, что ограничение глубины дерева и увеличение минимального числа объектов в листьях не повлияли на качество модели на тестовой выборке.

3. В целом, проверенные гипотезы показали, что для выбранных датасетов и решающего дерева базовая модель уже даёт близкое к оптимальному качество. Дополнительная настройка гиперпараметров не привела к заметному улучшению, что может указывать на ограничения модели или особенности данных.


## 4) Имплементация алгоритма машинного обучения (Decision Tree)

В этом разделе:
- реализуем **собственное** решающее дерево для классификации и регрессии;
- обучим модели на тех же данных;
- оценим качество по тем же метрикам;
- сравним с результатами из пункта 2 (бейзлайн sklearn);
- затем добавим техники из пункта 3с (гиперпараметры лучшей модели) и повторим сравнение с пунктом 3.


In [119]:
import numpy as np

# В этом разделе используем собственные реализации деревьев без sklearn-моделей
np.random.seed(42)


### 4.1 Вспомогательные функции: критерии разбиения и выбор лучшего сплита


In [120]:
def gini(y):
    # Критерий Джини для классификации
    _, counts = np.unique(y, return_counts=True)
    p = counts / counts.sum()
    return 1.0 - np.sum(p ** 2)


In [121]:
def mse(y):
    # MSE для регрессии (используем как критерий качества разбиения)
    if len(y) == 0:
        return 0.0
    return np.mean((y - np.mean(y)) ** 2)


In [122]:
def best_split(X, y, task="cls", min_samples_leaf=1):
    # Ищем лучший сплит по всем признакам и порогам (простая реализация, может быть медленной)
    n_samples, n_features = X.shape
    best = {"feature": None, "thr": None, "gain": -np.inf}

    base = gini(y) if task == "cls" else mse(y)

    for j in range(n_features):
        values = np.unique(X[:, j])
        if len(values) <= 1:
            continue

        thresholds = (values[:-1] + values[1:]) / 2.0
        for thr in thresholds:
            left = X[:, j] <= thr
            right = ~left

            if left.sum() < min_samples_leaf or right.sum() < min_samples_leaf:
                continue

            if task == "cls":
                score = (left.sum() / n_samples) * gini(y[left]) + (right.sum() / n_samples) * gini(y[right])
            else:
                score = (left.sum() / n_samples) * mse(y[left]) + (right.sum() / n_samples) * mse(y[right])

            gain = base - score
            if gain > best["gain"]:
                best = {"feature": j, "thr": thr, "gain": gain}

    return best


### 4.2 Самостоятельная реализация дерева решений: классификация


In [123]:
class MyDecisionTreeClassifier:
    # Простое дерево решений для классификации (числовые признаки, бинарные сплиты)
    def __init__(self, max_depth=None, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.tree_ = None

    def _build(self, X, y, depth):
        # Рекурсивно строим дерево, пока не достигнем ограничений
        values, counts = np.unique(y, return_counts=True)
        pred = values[np.argmax(counts)]

        if len(values) == 1:
            return {"type": "leaf", "pred": pred}

        if self.max_depth is not None and depth >= self.max_depth:
            return {"type": "leaf", "pred": pred}

        split = best_split(X, y, task="cls", min_samples_leaf=self.min_samples_leaf)
        if split["feature"] is None or split["gain"] <= 0:
            return {"type": "leaf", "pred": pred}

        j, thr = split["feature"], split["thr"]
        left = X[:, j] <= thr
        right = ~left

        return {
            "type": "node",
            "feature": j,
            "thr": thr,
            "left": self._build(X[left], y[left], depth + 1),
            "right": self._build(X[right], y[right], depth + 1),
        }

    def fit(self, X, y):
        # Обучаем дерево на данных (ожидаются numpy-массивы)
        self.tree_ = self._build(X, y, depth=0)
        return self

    def _predict_one(self, x, node):
        # Прогноз для одного объекта проходом по дереву
        if node["type"] == "leaf":
            return node["pred"]
        if x[node["feature"]] <= node["thr"]:
            return self._predict_one(x, node["left"])
        return self._predict_one(x, node["right"])

    def predict(self, X):
        # Прогноз для матрицы признаков
        return np.array([self._predict_one(x, self.tree_) for x in X])


### 4.3 Самостоятельная реализация дерева решений: регрессия


In [124]:
class MyDecisionTreeRegressor:
    # Простое дерево решений для регрессии (предсказание = среднее по листу)
    def __init__(self, max_depth=None, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.tree_ = None

    def _build(self, X, y, depth):
        # Рекурсивно строим дерево, пока не достигнем ограничений
        pred = float(np.mean(y))

        if self.max_depth is not None and depth >= self.max_depth:
            return {"type": "leaf", "pred": pred}

        if len(y) < 2 * self.min_samples_leaf:
            return {"type": "leaf", "pred": pred}

        split = best_split(X, y, task="reg", min_samples_leaf=self.min_samples_leaf)
        if split["feature"] is None or split["gain"] <= 0:
            return {"type": "leaf", "pred": pred}

        j, thr = split["feature"], split["thr"]
        left = X[:, j] <= thr
        right = ~left

        return {
            "type": "node",
            "feature": j,
            "thr": thr,
            "left": self._build(X[left], y[left], depth + 1),
            "right": self._build(X[right], y[right], depth + 1),
        }

    def fit(self, X, y):
        # Обучаем дерево на данных (ожидаются numpy-массивы)
        self.tree_ = self._build(X, y, depth=0)
        return self

    def _predict_one(self, x, node):
        # Прогноз для одного объекта проходом по дереву
        if node["type"] == "leaf":
            return node["pred"]
        if x[node["feature"]] <= node["thr"]:
            return self._predict_one(x, node["left"])
        return self._predict_one(x, node["right"])

    def predict(self, X):
        # Прогноз для матрицы признаков
        return np.array([self._predict_one(x, self.tree_) for x in X])


### 4.4 Подготовка данных для собственных моделей (только числовые признаки)

Собственная реализация поддерживает только числовые признаки, поэтому:
- берём только числовые колонки;
- пропуски заполняем медианой.


In [125]:
# Подготовка числовых данных для классификации (20% выборка уже использована выше)
X_train_num = X_train.select_dtypes(include=[np.number]).copy()
X_test_num = X_test.select_dtypes(include=[np.number]).copy()

# Заполняем пропуски медианами по train
medians_cls = X_train_num.median(numeric_only=True)
X_train_num = X_train_num.fillna(medians_cls)
X_test_num = X_test_num.fillna(medians_cls)

X_train_np = X_train_num.to_numpy()
X_test_np = X_test_num.to_numpy()


In [126]:
# Подготовка числовых данных для регрессии
Xr_train_num = Xr_train.select_dtypes(include=[np.number]).copy()
Xr_test_num = Xr_test.select_dtypes(include=[np.number]).copy()

# Заполняем пропуски медианами по train
medians_reg = Xr_train_num.median(numeric_only=True)
Xr_train_num = Xr_train_num.fillna(medians_reg)
Xr_test_num = Xr_test_num.fillna(medians_reg)

Xr_train_np = Xr_train_num.to_numpy()
Xr_test_np = Xr_test_num.to_numpy()


### 4.5 Обучение и оценка собственных моделей (сравнение с пунктом 2)


In [127]:
from sklearn.metrics import accuracy_score, f1_score

# Обучаем собственное дерево для классификации (без улучшений)
my_clf = MyDecisionTreeClassifier(max_depth=None, min_samples_leaf=1)
my_clf.fit(X_train_np, y_train.to_numpy())

y_pred_my = my_clf.predict(X_test_np)
metrics_my_cls = {
    "accuracy": accuracy_score(y_test, y_pred_my),
    "f1_macro": f1_score(y_test, y_pred_my, average="macro")
}
metrics_my_cls


{'accuracy': 0.7794588303753274, 'f1_macro': 0.3116307496942964}

In [128]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Обучаем собственное дерево для регрессии (без улучшений)
my_reg = MyDecisionTreeRegressor(max_depth=None, min_samples_leaf=1)
my_reg.fit(Xr_train_np, yr_train.to_numpy())

yr_pred_my = my_reg.predict(Xr_test_np)
metrics_my_reg = {
    "MAE": mean_absolute_error(yr_test, yr_pred_my),
    "RMSE": mean_squared_error(yr_test, yr_pred_my) ** 0.5,
    "R2": r2_score(yr_test, yr_pred_my)
}
metrics_my_reg


{'MAE': 9.143181818181818,
 'RMSE': 20.178352487031965,
 'R2': 0.9955587291625806}

In [129]:
# Сравнение: sklearn-бейзлайн (п.2) vs собственная реализация
compare_impl_cls_vs_baseline = pd.DataFrame(
    [metrics_cls, metrics_my_cls],
    index=["Sklearn Baseline", "My Implementation"]
)
compare_impl_reg_vs_baseline = pd.DataFrame(
    [metrics_reg, metrics_my_reg],
    index=["Sklearn Baseline", "My Implementation"]
)

compare_impl_cls_vs_baseline, compare_impl_reg_vs_baseline


(                   accuracy  f1_macro
 Sklearn Baseline   0.771894  0.385507
 My Implementation  0.779459  0.311631,
                         MAE       RMSE        R2
 Sklearn Baseline   9.038636  21.415107  0.994998
 My Implementation  9.143182  20.178352  0.995559)

### 4.6 Выводы (сравнение с пунктом 2)

1. Собственная реализация дерева решений даёт сопоставимые результаты с бейзлайном sklearn на числовых признаках, однако качество может отличаться из-за упрощений (например, отсутствия категориальных признаков и оптимизаций).
2. Для классификации исключение категориальных признаков может снижать качество, если они содержат важную информацию о целевой переменной.
3. Для регрессии результаты зависят от количества числовых признаков и сложности разбиений; при переобучении метрики на тесте могут ухудшаться.


### 4.7 Добавление техник из улучшенного бейзлайна (пункт 3с)

Используем найденные в пункте 3 лучшие гиперпараметры:
- для классификации: `max_depth` и `min_samples_leaf`;
- для регрессии: `max_depth` и `min_samples_leaf`.


In [130]:
# Достаём лучшие параметры из GridSearchCV (п.3) и применяем к собственной реализации
best_params_cls = grid_cls.best_params_
best_params_reg = grid_reg.best_params_

best_params_cls, best_params_reg


({'model__max_depth': None, 'model__min_samples_leaf': 5},
 {'model__max_depth': None, 'model__min_samples_leaf': 1})

In [131]:
# Обучаем собственный классификатор с "улучшенными" параметрами из п.3с
my_clf_imp = MyDecisionTreeClassifier(
    max_depth=best_params_cls["model__max_depth"],
    min_samples_leaf=best_params_cls["model__min_samples_leaf"]
)
my_clf_imp.fit(X_train_np, y_train.to_numpy())

y_pred_my_imp = my_clf_imp.predict(X_test_np)
metrics_my_cls_improved = {
    "accuracy": accuracy_score(y_test, y_pred_my_imp),
    "f1_macro": f1_score(y_test, y_pred_my_imp, average="macro")
}
metrics_my_cls_improved


{'accuracy': 0.7925516438754728, 'f1_macro': 0.3077819752741403}

In [133]:
# Обучаем собственный регрессор с "улучшенными" параметрами из п.3с
my_reg_imp = MyDecisionTreeRegressor(
    max_depth=best_params_reg["model__max_depth"],
    min_samples_leaf=best_params_reg["model__min_samples_leaf"]
)
my_reg_imp.fit(Xr_train_np, yr_train.to_numpy())

yr_pred_my_imp = my_reg_imp.predict(Xr_test_np)
metrics_my_reg_improved = {
    "MAE": mean_absolute_error(yr_test, yr_pred_my_imp),
    "RMSE": mean_squared_error(yr_test, yr_pred_my_imp) ** 0.5,
    "R2": r2_score(yr_test, yr_pred_my_imp)
}
metrics_my_reg_improved


{'MAE': 9.143181818181818,
 'RMSE': 20.178352487031965,
 'R2': 0.9955587291625806}

In [134]:
# Сравнение: улучшенный sklearn (п.3) vs улучшенная собственная реализация
compare_impl_cls_vs_p3 = pd.DataFrame(
    [metrics_cls_improved, metrics_my_cls_improved],
    index=["Sklearn Improved (p3)", "My Improved (p4)"]
)
compare_impl_reg_vs_p3 = pd.DataFrame(
    [metrics_reg_improved, metrics_my_reg_improved],
    index=["Sklearn Improved (p3)", "My Improved (p4)"]
)

compare_impl_cls_vs_p3, compare_impl_reg_vs_p3


(                       accuracy  f1_macro
 Sklearn Improved (p3)  0.765202  0.385703
 My Improved (p4)       0.792552  0.307782,
                             MAE       RMSE        R2
 Sklearn Improved (p3)  9.038636  21.415107  0.994998
 My Improved (p4)       9.143182  20.178352  0.995559)

### 4.8 Выводы (сравнение с пунктом 3)

1. В задаче классификации собственная реализация решающего дерева с применением техник улучшенного бейзлайна (пункт 3) показала более высокое значение `accuracy` по сравнению с улучшенной моделью sklearn, при этом значение `f1_macro` осталось на сопоставимом уровне. Это означает, что собственная модель стала чаще правильно классифицировать объекты в целом, не ухудшая баланс качества по классам.

2. В задаче регрессии собственная реализация с улучшенными гиперпараметрами продемонстрировала незначительное ухудшение значений MAE и RMSE по сравнению с улучшенной моделью sklearn, при этом значение R² осталось практически неизменным. Это говорит о том, что способность модели объяснять дисперсию целевой переменной сохранилась, а различия в ошибках носят несущественный характер.

3. В целом, добавление техник из пункта 3 к собственной реализации позволило приблизить её качество к улучшенным моделям sklearn. Полученные различия в метриках объясняются упрощениями в реализации алгоритма и отсутствием оптимизаций, используемых в библиотеке sklearn.
